In [1]:
import threading
from queue import Queue, Empty
from PIL import Image, ImageFilter
import os

class ImageProcessor:
    def __init__(self, input_folder, output_folder, num_threads=4):
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.num_threads = num_threads
        self.queue = Queue()
        self.lock = threading.Lock()

    def process_images(self):
        # Create output folder if it doesn't exist
        if not os.path.exists(self.output_folder):
            os.makedirs(self.output_folder)
            print(f"Created output folder: {self.output_folder}")

        # Populate queue with image files
        image_count = 0
        for filename in os.listdir(self.input_folder):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                self.queue.put(filename)
                image_count += 1

        print(f"Found {image_count} images to process.")

        if image_count == 0:
            print("No images found in the input folder.")
            return

        # Create and start worker threads
        threads = []
        for i in range(self.num_threads):
            thread = threading.Thread(target=self.worker, name=f"Worker-{i+1}")
            thread.start()
            threads.append(thread)
            print(f"Started thread {thread.name}")

        # Wait for all threads to complete
        for thread in threads:
            thread.join()
            print(f"Thread {thread.name} has completed.")

    def worker(self):
        while True:
            try:
                # Get an image filename from the queue
                filename = self.queue.get(block=False)
            except Empty:  # Corrected exception handling
                break

            input_path = os.path.join(self.input_folder, filename)
            output_path = os.path.join(self.output_folder, f"processed_{filename}")

            try:
                # Open the image
                with Image.open(input_path) as img:
                    # Apply a series of filters
                    img = img.filter(ImageFilter.SHARPEN)
                    img = img.filter(ImageFilter.EDGE_ENHANCE)
                    img = img.filter(ImageFilter.SMOOTH)

                    # Save the processed image
                    img.save(output_path)

                with self.lock:
                    print(f"Processed: {filename}")

            except Exception as e:
                with self.lock:
                    print(f"Error processing {filename}: {str(e)}")

            self.queue.task_done()

if __name__ == "__main__":
    input_folder = "input_images"
    output_folder = "output_images"

    # Create the input folder if it doesn't exist
    if not os.path.exists(input_folder):
        os.makedirs(input_folder)
        print(f"Created input folder: {input_folder}. Please add images to process.")

    processor = ImageProcessor(input_folder, output_folder, num_threads=4)
    processor.process_images()


Created input folder: input_images. Please add images to process.
Created output folder: output_images
Found 0 images to process.
No images found in the input folder.
